In [1]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn as skl
from sklearn.cluster import KMeans
# %load_ext line_profiler

%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (16.0, 8.0)

Populating the interactive namespace from numpy and matplotlib


C:\Anaconda3\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Parameters

In [2]:
n = 500000  #  num of random instances to train on
m = 100  #  num of landmarks to use in Subset of Regressors

# Load Data

In [3]:
original_store_info_data = pd.read_csv('kaggleData/store_info.csv')
original_store_test_data = pd.read_csv('kaggleData/store_test.csv')
original_store_train_data = pd.read_csv('kaggleData/store_train.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#  Clean data
Convert all data to use Month # instead of Date
change all values to numerical format

In [4]:
def extract_month(date):
    return int(date.split("-")[1])

date_col = original_store_test_data['Date']
month_col = date_col.map(extract_month)
type(month_col)
original_store_test_data['Month'] = month_col
del(original_store_test_data['Date'])

date_col = original_store_train_data['Date']
month_col = date_col.map(extract_month)
type(month_col)
original_store_train_data['Month'] = month_col
del(original_store_train_data['Date'])
del(original_store_train_data['Customers'])


In [5]:
reduced_indexes = np.random.choice(original_store_train_data.index, n, replace=False)
store_train_data_reduced = original_store_train_data.loc[reduced_indexes]

store_train_data_reduced_labels = store_train_data_reduced['Sales']
del(store_train_data_reduced['Sales'])


In [6]:
def convert_hol_to_numeric(x):
    if x == "a":
        return 0
    elif x == "b":
        return 1
    elif x == "c":
        return 2
    else:
        return 3

store_train_data_reduced['StateHoliday'] = store_train_data_reduced['StateHoliday'].map(convert_hol_to_numeric) 
original_store_test_data['StateHoliday'] = original_store_test_data['StateHoliday'].map(convert_hol_to_numeric) 

In [7]:

store_train_data_reduced.head()

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Month
98580,461,1,1,1,3,0,5
530183,229,4,1,0,3,0,3
519282,478,7,0,0,3,0,3
643803,119,7,0,0,3,0,12
593049,655,4,1,0,3,0,1


In [8]:
original_store_train_data.describe()


,Store,DayOfWeek,Sales,Open,Promo,SchoolHoliday,Month
count,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000
mean,558.429727,3.998341,5773.818972,0.830107,0.381515,0.178647,5.846762
std,321.908651,1.997391,3849.926175,0.375539,0.485759,0.383056,3.326097
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,280.000000,2.000000,3727.000000,1.000000,0.000000,0.000000,3.000000
50%,558.000000,4.000000,5744.000000,1.000000,0.000000,0.000000,6.000000
75%,838.000000,6.000000,7856.000000,1.000000,1.000000,0.000000,8.000000
max,1115.000000,7.000000,41551.000000,1.000000,1.000000,1.000000,12.000000


In [9]:
original_store_train_data.head(2)

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,Month
0,1,5,5263,1,1,0,1,7
1,2,5,6064,1,1,0,1,7


In [10]:
original_store_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 8 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Sales            1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
Month            1017209 non-null int64
dtypes: int64(7), object(1)
memory usage: 69.8+ MB


In [11]:
original_store_test_data.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Month
0,1,1,4,1,1,3,0,9
1,2,3,4,1,1,3,0,9
2,3,7,4,1,1,3,0,9
3,4,8,4,1,1,3,0,9
4,5,9,4,1,1,3,0,9


#  Define Kernel methods for evaluating the covariance / distance between two data points
# Subset of Regressors Model
*A data point is one store's statistics for one day. 

In [12]:
class Subset_Of_Regressors():
    def __init__(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train
        
        self.__initialize_kernels()
        

        self.sigma2 = 1.0
#         self.y =   #  who knows?
    
    
    def __initialize_kernels(self):
        #  initliaze matrix for dow
        self.dow_cov_matrix = np.zeros((7,7))
        for i in np.arange(7):
            self.dow_cov_matrix[i][i] = 1.0 
            self.dow_cov_matrix[i][(i + 1) % 7] = 0.5 
            self.dow_cov_matrix[i][(i + 2) % 7] = 0.1 
            self.dow_cov_matrix[i][(i - 1) % 7] = 0.5 
            self.dow_cov_matrix[i][(i - 2) % 7] = 0.1 
        self.dow_cov_matrix = np.dot(self.dow_cov_matrix, self.dow_cov_matrix.T)

        #  initlize matrix for state holidays
        self.state_hol_cov_matrix = np.zeros((4,4))
        self.state_hol_cov_matrix[0][0] = 1.0   
        self.state_hol_cov_matrix[1][1] = 1.0       
        self.state_hol_cov_matrix[2][2] = 1.0       
        self.state_hol_cov_matrix[3][3] = 1.0       
        self.state_hol_cov_matrix[2][1] = 0.3       
        self.state_hol_cov_matrix[1][2] = 0.3       
        self.state_hol_cov_matrix[0][2] = 0.2       
        self.state_hol_cov_matrix[2][0] = 0.2       
        self.state_hol_cov_matrix[0][1] = 0.5       
        self.state_hol_cov_matrix[1][0] = 0.5   
        self.state_hol_cov_matrix = np.dot(self.state_hol_cov_matrix, self.state_hol_cov_matrix.T)
        
        #  initilize matrx for month
        self.month_cov_matrix = np.zeros((12,12))
        for i in np.arange(12):
            self.month_cov_matrix[i][i] = 1.0 
            self.month_cov_matrix[i][(i + 1) % 12] = 0.5 
            self.month_cov_matrix[i][(i + 2) % 12] = 0.3 
            self.month_cov_matrix[i][(i + 3) % 12] = 0.1 
            self.month_cov_matrix[i][(i - 1) % 12] = 0.5 
            self.month_cov_matrix[i][(i - 2) % 12] = 0.3 
            self.month_cov_matrix[i][(i - 3) % 12] = 0.1 
        self.month_cov_matrix = np.dot(self.month_cov_matrix, self.month_cov_matrix.T)
    
    
    #  accepts two different store points (column vectors) and returns their covariance distance
    def __rossman_kernel(self, v1, v2):
        kernel_weights = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

        return kernel_weights[0] * self.__store_kernel(v1[0], v2[0]) + kernel_weights[1] * self.__day_of_week_kernel(v1[1], v2[1]) \
                + kernel_weights[2] * self.__open_kernel(v1[2], v2[2]) \
                + kernel_weights[3] * self.__promo_kernel(v1[3], v2[3]) + kernel_weights[4] * self.__state_hol_kernel(v1[4], v2[4]) \
                + kernel_weights[5] * self.__school_hol_kernel(v1[5], v2[5]) + kernel_weights[6] * self.__month_kernel(v1[6], v2[6])
                

    def __store_kernel(self, x1, x2):
        if(x1 == x2):
            return 1.0
        else:
            return 0.0

    def __day_of_week_kernel(self, x1, x2):
        return self.dow_cov_matrix[x1 - 1,x2 - 1] / 1.52  # normalizing for dow matrix

    def __month_kernel(self, x1, x2):
        return self.month_cov_matrix[x1 - 1,x2 - 1] / 1.7  #  normalizing for month matrix

    def __open_kernel(self, x1, x2):
        if(x1 == x2):
            return 1.0
        else:
            return 0.0

    def __promo_kernel(self, x1, x2):
        if(x1 == x2):
            return 1.0
        else:
            return 0.0

    def __state_hol_kernel(self, x1, x2):
        return self.state_hol_cov_matrix[x1,x2] / 1.34 # normlaizing for state hol matrix
    
    def __school_hol_kernel(self, x1, x2):
        if(x1 == x2):
            return 1.0
        else:
            return 0.0

        
    def __select_m_random(self, matrix, m):
        return matrix[np.random.choice(len(matrix), m, replace=False)]


    def __select_m_means(self, matrix, m):
        model = KMeans(m, precompute_distances=True, n_jobs=8)
        model.fit(matrix)
        landmarks = model.cluster_centers_
        def to_int(x):
            return int(x)
        vec_to_int = np.vectorize(to_int)
        return [ vec_to_int(i) for i in landmarks]
    
    
    def train(self, m, n):
        print("Starting Training")
        self.m = m
        self.n = n
        
        self.k_mm = np.zeros((m,m))
        self.k_mn = np.zeros((m,n))
        self.k_m = np.zeros((1,m))
        
        self.landmarks = self.__select_m_means(self.x_train, self.m)
        
        self.__build_matricies()
        self.combined_matrix = self.__precompute_combined_matrix()
    
    def __build_matricies(self):
        #  set up mm
        for i in range(self.m):
            print("%d / %d" % (i, self.m), end="\r")

            for j in range(self.m):
                self.k_mm[i][j] = self.__rossman_kernel(self.landmarks[i], self.landmarks[j])
                
        #  set up mn
        for i in range(self.m):
            for j in range(len(self.x_train)):
                self.k_mn[i][j] = self.__rossman_kernel(self.landmarks[i], self.x_train[j])
    
    def __precompute_combined_matrix(self):
        tmp_matrix = (np.dot(self.k_mn, self.k_mn.T) + self.sigma2 * self.k_mm)
        tmp_matrix = np.linalg.pinv(tmp_matrix)
        tmp_matrix = np.dot(tmp_matrix, self.k_mn)
        return np.dot(tmp_matrix, self.y_train)
            
    def display_mm(self):
        sns.heatmap(self.k_mm, fig)
        
    def classify(self, x_test):
        k_tm = np.zeros((1,self.m))
        k_tm[0] = [self.__rossman_kernel(x_test, landmark) for landmark in self.landmarks]
        
        return np.dot(k_tm, self.combined_matrix)[0]
    
    def classify_all(self, all_x_test):
        print("Starting Classification")
        classifications = []
        
        all_x_test_size = len(all_x_test)
        for i, x_test in enumerate(all_x_test):
            print("%d / %d" % (i, all_x_test_size), end="\r")

            classifications.append((x_test[0], self.classify(x_test[1:])))
            
        return classifications
        

#  Display MxM Kernel Matrix

In [13]:
%time model = Subset_Of_Regressors(store_train_data_reduced.values.astype('int64'), store_train_data_reduced_labels.values.astype('int64'))



Wall time: 50.1 ms


In [14]:
%time model.train(m, n)


Starting Training
Wall time: 11min 6s


In [15]:
%time model.display_mm()

NameError: name 'fig' is not defined

In [16]:
%time classifications = model.classify_all(original_store_test_data.values)
%time classifications

Starting Classification
Wall time: 44.8 s
Wall time: 0 ns


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:65: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:83: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:68: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[(1.0, 3426.6254394967309),
 (2.0, 3426.6254394967309),
 (3.0, 3426.6254394967309),
 (4.0, 3426.6254394967309),
 (5.0, 3426.6254394967309),
 (6.0, 3426.6254394967309),
 (7.0, 3426.6254394967309),
 (8.0, 3426.6254394967309),
 (9.0, 3426.6254394967309),
 (10.0, 2496.6178608076207),
 (11.0, 3426.6254394967309),
 (12.0, 3426.6254394967309),
 (13.0, 3426.6254394967309),
 (14.0, 3426.6254394967309),
 (15.0, 3426.6254394967309),
 (16.0, 3426.6254394967309),
 (17.0, 3426.6254394967309),
 (18.0, 5340.3290231932551),
 (19.0, 3426.6254394967309),
 (20.0, 3426.6254394967309),
 (21.0, 3426.6254394967309),
 (22.0, 3426.6254394967309),
 (23.0, 3426.6254394967309),
 (24.0, 3426.6254394967309),
 (25.0, 3426.6254394967309),
 (26.0, 3426.6254394967309),
 (27.0, 3426.6254394967309),
 (28.0, 3426.6254394967309),
 (29.0, 3426.6254394967309),
 (30.0, 3426.6254394967309),
 (31.0, 3426.6254394967309),
 (32.0, 3426.6254394967309),
 (33.0, 3426.6254394967309),
 (34.0, 3426.6254394967309),
 (35.0, 3426.6254394967

In [17]:
sfile = open( 'mean_sub_2.csv', 'w' )
sfile.write( '"Id","Sales"\n' )
for classification in classifications:
    sfile.write( '%d,%.2f\n' % ( classification[0], classification[1] ) )  # add one for one-based indexing
sfile.close()

#  Kaggle Results
## I was rank 3173 with an over MSE of 0.52023 on my Kaggle submission.